In [1]:
!pip install matplotlib

Configuracion del puerto en serie

In [2]:
import serial, time
import numpy as np
import matplotlib.pyplot as plt

serial_port = 'COM4'  # Cambia según tu puerto
bauds = 250000


Definimos parametros

In [3]:
label = 'letter_t'
gestures_per_label = 10
gesture_duration = 2.0
sample_frequency = 100
samples_number = int(sample_frequency * gesture_duration)


Capturamos una muestra

In [4]:
ser = serial.Serial(serial_port, bauds, timeout=1.0)
time.sleep(2)
ser.write('r'.encode('ascii'))
print(ser.readline())
ser.close()


b'0.02\t0.25\t0.94\r\n'


Definimos función

In [5]:
def capture_gesture():
    ser = serial.Serial(serial_port, bauds, timeout=1.0)
    time.sleep(2)
    gesture = []
    for _ in range(samples_number):
        ser.write('r'.encode('ascii'))
        line = ser.readline().decode().strip()
        if line:
            values = line.split('\t')
            if len(values) == 3:
                gesture.append([float(v) for v in values])
        time.sleep(1 / sample_frequency)
    ser.close()
    return np.array(gesture)


Capturamos varios gestos

In [ ]:
gestures = []
for i in range(gestures_per_label):
    input(f'Pulsa Enter para grabar gesto {i+1}')
    g = capture_gesture()
    gestures.append(g)


Filtro para suavizar señales

In [ ]:
from scipy.signal import butter, filtfilt

def butter_lowpass_filter(data, cutoff=3, fs=100, order=6):
    b, a = butter(order, cutoff / (0.5 * fs), btype='low')
    return filtfilt(b, a, data, axis=0)

gestures_filtered = [butter_lowpass_filter(g) for g in gestures]


Extracción de características

In [ ]:
from scipy.fft import fft

def get_spectrum(gesture):
    return np.abs(fft(gesture, axis=0))

spectrums = [get_spectrum(g) for g in gestures_filtered]


Exportacion CSV

In [ ]:
import pandas as pd

for idx, g in enumerate(gestures_filtered):
    df = pd.DataFrame(g, columns=['X', 'Y', 'Z'])
    df.to_csv(f'gesture_{label}_{idx+1}.csv', index=False)
